## Global planning test

Test global costmap generation and global planners (A*, D*) 

In [2]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import cv2 as cv
from mpl_toolkits.axes_grid1 import make_axes_locatable
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

from terrain_nerf.global_planner import GlobalPlanner
from terrain_nerf.utils import rgb2gray
from terrain_nerf.feature_map import FeatureMap, CostMap
from terrain_nerf.autonav import AutoNav

# autoreload
%load_ext autoreload
%autoreload 2

### Global planner

In [3]:
global_img = cv.imread('../data/airsim/images/moonenv/global_map.png')
fig = px.imshow(global_img)
fig.update_layout(width=1600, height=800)
fig.show()

In [4]:
# downsample image by 2
global_img = global_img[::2, ::2, :]
fig = px.imshow(global_img)
fig.update_layout(width=1600, height=800)
fig.show()

In [ ]:
# xy and reversed with respect to plotly image coordinates
start_px = (171, 751)
goal_px = (238, 117)

Segment anything

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
sam_checkpoint = "../data/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
masks = mask_generator.generate(global_img)

In [ ]:
plt.figure(figsize=(15,8))
plt.imshow(global_img)
show_anns(masks)
plt.axis('off')
plt.show() 

In [ ]:
plt.imshow(masks[2]['segmentation'])
plt.show()

Initialize costmap

In [ ]:
costmap_mat = np.zeros((global_img.shape[0], global_img.shape[1]))
costmap_clusters = np.zeros((global_img.shape[0], global_img.shape[1]))
costmap_vals = np.zeros(len(masks))
for i, ann in enumerate(masks):
    costmap_clusters[ann['segmentation']] = i
    costmap_vals[i] = 0.1 * global_img[ann['segmentation']].mean() / 255.0
    costmap_mat[ann['segmentation']] = costmap_vals[i]

costmap = CostMap(costmap_mat, costmap_clusters, costmap_vals)
np.savez_compressed('../data/airsim/costmap.npz', mat=costmap_mat, clusters=costmap_clusters, vals=costmap_vals)

Path planning

In [ ]:
UNREAL_PLAYER_START = np.array([-117252.054688, 264463.03125, 25148.908203])
UNREAL_GOAL = np.array([-83250.0, 258070.0, 24860.0])
GOAL_POS = (UNREAL_GOAL - UNREAL_PLAYER_START)[:2] / 100.0

feat_map = FeatureMap(global_img, start_px, goal_px, UNREAL_PLAYER_START, UNREAL_GOAL)

In [ ]:
# Initialize costmap
gp = GlobalPlanner(costmap, feat_map, goal_px)

# Show image and cost side by side
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(global_img)
im = ax[1].imshow(gp.costmap.mat)
plt.colorbar(im, ax=ax[1], fraction=0.05, aspect=10)
plt.show()

In [ ]:
path = gp.replan(np.zeros(3))
# Plot 
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
gp.plot(ax=ax)
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.show()

In [ ]:
# gp.costmap[costmap_clusters==3] = 0.5
# path = gp.plan(start_px, goal_px)
# # Plot 
# fig, ax = plt.subplots(1, 1, figsize=(15, 5))
# ax.imshow(costmap, cmap='viridis')
# ax.plot(path[:, 1], path[:, 0], 'r')
# plt.show()

Update costmap (Naive)

In [ ]:
depth = np.load('../data/airsim/images/depth_1688174586.6823223.npy')
pose = np.array([169.813, -23.864, -0.104])  # x, y, yaw

depth_vis = depth.copy()
depth_vis[depth_vis > 20] = 20.0

autonav = AutoNavDepth(np.zeros(2))
cost_vals = autonav.update_costmap(pose, depth)
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(depth_vis)
im = autonav.plot_costmap(ax[1])
plt.colorbar(im)

In [ ]:
cvs = np.array(cost_vals)
plt.scatter(cvs[:, 1], cvs[:, 0], c=cvs[:, 2], cmap='viridis')

In [ ]:
costs = cvs[:,2]
costs = costs[costs > 1e-3]
costs.mean()

In [ ]:
gp.update_costmap(cost_vals)
path = gp.plan(start_px, goal_px)
# Plot 
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.imshow(gp.costmap.mat, cmap='viridis')
ax.plot(path[:, 1], path[:, 0], 'r')
plt.show()